In [58]:
import os
from itertools import product
from collections import namedtuple
import time
from typing import Tuple
import random

from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import patches
import numpy as np
import pandas as pd

# Подключаем локальные пакеты
from pyqumo.random import Distribution, Exponential, HyperExponential, Erlang
from pyqumo.cqumo.sim import simulate_tandem


from pyqumo.fitting.johnson89 import fit_mern2
from pyqumo.stats import get_cv, get_skewness, get_noncentral_m2, get_noncentral_m3
from pyqumo.random import HyperErlang

# Поключаем пакеты для ML
import math
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from catboost import CatBoostClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from tensorflow.python.keras import engine
from tensorflow.keras import layers
import seaborn as sns
from scipy import stats

In [59]:
def get_std(m1, m2):
    return (m2 - m1**2)**0.5

In [60]:
# Нужно ли пересчитывать все, или можно использовать результаты из файлов
FORCE_SIMULATION = True
# SIM_FILE_NAME = '01_tandem_simulation.csv'
SIM_FILE_NAME = 'Tandem_network_with_m_ph_1_n_distribution.csv'
SIM_FILE_DIR = 'data'
SIM_FILE_PATH = os.path.join(SIM_FILE_DIR, SIM_FILE_NAME)

# Зададим число пакетов, передачу которых по сети мы будем моделировать.
# Чем выше это число, тем точнее результаты, но на их получение нужно больше времени.
NUM_PACKETS = 100000

# Цветовая схема для графиков
CMAP_NAME = 'viridis'

In [75]:
COLUMNS = (
    'ArrM1',
    'ArrAvg',
    'SrvM1',
    'SrvM2',
    'SrvM3',
    'SrvAvg', 
    'SrvStd', 
    'SrvCv',
    'SrvSkewness', 
    'Rho', 
    'NetSize', 
    'Capacity', 
    'NumPackets',
    'DelayAvg', 
    'DelayStd', 
    'DeliveryProb',
)


def save_sim_data(df: pd.DataFrame, ):
    """
    Сохранить в файл данные о результатах имитационного моделирования.
    """
    if not os.path.exists(SIM_FILE_DIR):
        os.makedirs(SIM_FILE_DIR)
    df.to_csv(SIM_FILE_PATH, index_label='Id')

    
def load_sim_data() -> pd.DataFrame:
    """
    Загрузить данные о резулдьтатах имитационного моделирования.
    """       
    if os.path.exists(SIM_FILE_PATH):
        return pd.read_csv(SIM_FILE_PATH, index_col='Id')
    return pd.DataFrame(columns=COLUMNS, index_col='Id')

sim_data = load_sim_data()
sim_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ArrM1         0 non-null      object
 1   ArrAvg        0 non-null      object
 2   SrvM1         0 non-null      object
 3   SrvM2         0 non-null      object
 4   SrvM3         0 non-null      object
 5   SrvAvg        0 non-null      object
 6   SrvStd        0 non-null      object
 7   SrvCv         0 non-null      object
 8   SrvSkewness   0 non-null      object
 9   Rho           0 non-null      object
 10  NetSize       0 non-null      object
 11  Capacity      0 non-null      object
 12  NumPackets    0 non-null      object
 13  DelayAvg      0 non-null      object
 14  DelayStd      0 non-null      object
 15  DeliveryProb  0 non-null      object
dtypes: object(16)
memory usage: 0.0+ bytes


In [76]:
# Simulation of queueing model M/PH/1/N -> ... -> ./PH/1/N.
def simulate(
    df: pd.DataFrame, *,
    arr_m1: float,        
    srv_m1: float,
    srv_m2: float,
    srv_m3: float,
    net_size: int,
    capacity: int,
    num_packets: int,
    force: bool = False
) -> pd.DataFrame:
    """
    Выполнить симуляцию, если результатов нет в `df` или требуется их пересчитать, и вернуть новый `DataFrame`.
    """
    row_df = df[
        (df.ArrM1 == arr_m1) &
        (df.ArrAvg == arr_m1) &
        (df.SrvM1 == srv_m1) &
        (df.SrvM2 == srv_m2) &
        (df.SrvM3 == srv_m3) &
        (df.NetSize == net_size) &
        (df.Capacity == capacity)]
    
    # Вычислим признаки, которые говорят о необходимости пересчета:
    no_row = len(row_df) == 0
    not_enough_packets = (not no_row) and (row_df.NumPackets.iloc[0] < num_packets)

    # Проверим, нужно ли пересчитать результаты:
    if force or no_row or not_enough_packets: 
        # Exponential distribution of interarrival packet time. 
        arr = Exponential(1/arr_m1)
        
        # Phase-type distribution of service time distribution.
        srv,_ = fit_mern2(moments=[srv_m1, srv_m2, srv_m3])
        
        ret = simulate_tandem([arr] * net_size, [srv] * net_size, capacity, num_packets)

        row_data = {
            'ArrM1': arr_m1,
            'ArrAvg': arr_m1,
            'SrvM1': srv_m1,
            'SrvM2': srv_m2,
            'SrvM3': srv_m3,
            'SrvAvg': srv_m1,
            'SrvStd': get_std(srv_m1, srv_m2),
            'SrvCv': get_cv(srv_m1, srv_m2),
            'SrvSkewness': get_skewness(srv_m1, srv_m2, srv_m3),
            'Rho': ret.get_utilization(net_size-1),
            'NetSize': net_size,
            'Capacity': capacity,
            'NumPackets': num_packets,
            'DelayAvg': ret.delivery_delays[0].avg,
            'DelayStd': ret.delivery_delays[0].std,
            'DeliveryProb': ret.delivery_prob[0],
        }
        # Если строки еще вообще не было, добавляем ее, а если была - обновляем:
        if no_row:
            df = df.append(row_data, ignore_index=True)
        else:
            df.update(pd.DataFrame(row_data, index=[row_df.index[0]]))

    return df

In [83]:
SAMPLE_NUM = 10_000
arr_m1 = np.zeros(SAMPLE_NUM)

srv_m1 = np.zeros(SAMPLE_NUM)
srv_cv = np.zeros(SAMPLE_NUM)
srv_m2 = np.zeros(SAMPLE_NUM)
srv_skewness = np.zeros(SAMPLE_NUM)
srv_m3 = np.zeros(SAMPLE_NUM)

net_size = np.zeros(SAMPLE_NUM)

time_scale = 1000

for i in range(SAMPLE_NUM):
#     arr_m1[i] = np.random.uniform(0.01 * time_scale, 0.05* time_scale)
    arr_m1[i] = np.random.uniform(10, 50)

    srv_m1[i] = np.random.uniform(0.5, 1)
    srv_cv[i] = np.random.uniform(4.45, 8)
    srv_m2[i] = get_noncentral_m2(srv_m1[i], srv_cv[i])
    srv_skewness[i] = np.random.uniform(srv_cv[i] - 1/srv_cv[i], 150)
    srv_m3[i] = get_noncentral_m3(srv_m1[i], srv_cv[i], srv_skewness[i])

    net_size[i] = np.random.randint(1, 20+1)
    
capacity = 20

In [84]:
  srv_m1[0]

0.7066412557840305

In [85]:
    srv_m1[0]/srv_m2[0]

0.027316232047921957

In [86]:
time_scale = 10
m1 = 0.49140462553574354
m2 = 5.043670174278639
m1, m2


(0.49140462553574354, 5.043670174278639)

In [87]:
get_cv(m1, m2)

4.45944184614579

In [88]:
if FORCE_SIMULATION:
    for i in tqdm(range(SAMPLE_NUM)):
        sim_data = simulate(
            sim_data,
            arr_m1=arr_m1[i],
            srv_m1=srv_m1[i],
            srv_m2=srv_m2[i],
            srv_m3=srv_m3[i],
            net_size=int(net_size[i]),
            capacity=capacity,
            num_packets=NUM_PACKETS,
            force=FORCE_SIMULATION
        )

    print(sim_data.info())
    print(sim_data)

    # Сохраняем результат:
    save_sim_data(sim_data)
else:
    print("Going to use previously computed results. To re-run simulation, set FORCE_SIMULATION = True")

  0%|          | 0/10000 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10010 entries, 0 to 10009
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ArrM1         10010 non-null  float64
 1   ArrAvg        10010 non-null  float64
 2   SrvM1         10010 non-null  float64
 3   SrvM2         10010 non-null  float64
 4   SrvM3         10010 non-null  float64
 5   SrvAvg        10010 non-null  float64
 6   SrvStd        10010 non-null  float64
 7   SrvCv         10010 non-null  float64
 8   SrvSkewness   10010 non-null  float64
 9   Rho           10010 non-null  float64
 10  NetSize       10010 non-null  float64
 11  Capacity      10010 non-null  float64
 12  NumPackets    10010 non-null  float64
 13  DelayAvg      10010 non-null  float64
 14  DelayStd      10010 non-null  float64
 15  DeliveryProb  10010 non-null  float64
dtypes: float64(16)
memory usage: 1.2 MB
None
           ArrM1     ArrAvg     SrvM1      SrvM2         SrvM3    SrvAv

In [ ]:
1/ 0.02

In [114]:
1 / 0.0005

2000.0

In [115]:
0.0005/ 0.002

0.25

In [110]:
arr_m1
srv_m1

array([0.00023479, 0.00028875, 0.00027234, 0.00025786, 0.00041654,
       0.0002624 , 0.0004242 , 0.00028743, 0.00027334, 0.00034382])

In [20]:
m1 = 0.0004914046255357437
m2 = 5.043670174278637e-06
m3 = 1.2319950843773457e-06

In [54]:
c= get_cv(m1, m2)
get_cv(m1, m2)

4.459441846145787

In [22]:
get_skewness(m1, m2, m3)

116.3871050683068

In [24]:
c - 1/c


4.23519853622029

In [53]:
np.random.uniform(0, 0.0005)

0.00036939742346539837

In [17]:
1492 * 8


11936

In [22]:
54 * 10**6

54000000

In [23]:
54000000/11936

4524.128686327078

In [34]:
1/4524.

0.00022104332449160034